In [2]:
from PIL import Image
from PIL.ExifTags import TAGS
import os
import csv
import pandas as pd
import numpy as np
import shutil
import const


C:\Users\Reizak\AppData\Local\Temp\ipykernel_2916\2845625259.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [74]:
#Fonction Recuperant des metadata d'une image 
def getImagesInfos(imagePath):
    image = Image.open(imagePath)
    info_dict = {
    "Filename": image.filename,
    "Size": image.size,
    "Height": image.height,
    "Width": image.width,
    "Format": image.format,
    "Mode": image.mode,}
    return info_dict


In [3]:
#Fonction générant un csv qui présente les metadatas de trois set 
def generate_metadata_csv(filename = "imagesInfos.csv", dataset_path = const.DATASET_PATH):
    with open(os.path.join(const.DATA_PATH, filename), 'w', newline='') as f:
        writer = csv.writer(f, delimiter=",")
        writer.writerow(["set", "birdName", "filename", "size", "height", "width", "format", "mode"])
        for setPath in os.listdir(dataset_path):
            fullSetPath = os.path.join(dataset_path, setPath)
            for birdPath in os.listdir(fullSetPath): 
                birdImagesList = os.listdir(os.path.join(fullSetPath, birdPath))
                for file in birdImagesList:
                    infos = getImagesInfos(os.path.join(fullSetPath, birdPath, file,))
                    writer.writerow([setPath, birdPath, file, infos['Size'], infos['Height'], infos['Width'], infos['Format'], infos['Mode']])

In [76]:
#Génération du csv de metadata pour le dossier d'origine
generate_metadata_csv()

In [4]:
#Informations sur la taille des images
df = pd.read_csv(os.path.join(const.DATA_PATH, "imagesInfos.csv"))
print("Nombre de taille d'images différentes : " + str(df['size'].nunique()))
print('\n')
print("Nombre de taille d'images différentes dans le set de test : " + str(df[df['set']== "test"]['size'].nunique()))
print("Nombre de taille d'images différentes dans le set d'entrainement : " + str(df[df['set']== "train"]['size'].nunique()))
print("Nombre de taille d'images différentes dans le set de validation : " + str(df[df['set']== "valid"]['size'].nunique()))
print('\n')

print("Nombre d'images de taille différente de 224*224 dans le set de test : " + str(df[(df['set']== "test") & (df['size'] != '(224, 224)')]['size'].nunique()))
print("Nombre d'images de taille différente de 224*224 dans le set d'entrainement : " + str(df[(df['set']== "train") & (df['size'] != '(224, 224)')]['size'].nunique()))
print("Nombre d'images de taille différente de 224*224 dans le set de validation : " + str(df[(df['set']== "valid") & (df['size'] != '(224, 224)')]['size'].nunique()))

#On voudra remplacer tout ça par des graphes


Nombre de taille d'images différentes : 212


Nombre de taille d'images différentes dans le set de test : 6
Nombre de taille d'images différentes dans le set d'entrainement : 202
Nombre de taille d'images différentes dans le set de validation : 6


Nombre d'images de taille différente de 224*224 dans le set de test : 5
Nombre d'images de taille différente de 224*224 dans le set d'entrainement : 201
Nombre d'images de taille différente de 224*224 dans le set de validation : 5


In [5]:
#On extrait seulement les lignes montrant une image qui n'est pas aux bonnes dimensions
#Et on les groupe par classe
df_to_resize = df[df['size'] != '(224, 224)']
df_to_resize.groupby(['birdName', 'set']).count()

filename  size  height  width  format  mode
birdName          set                                               
LOGGERHEAD SHRIKE train         1     1       1      1       1     1
PLUSH CRESTED JAY test          5     5       5      5       5     5
                  train       200   200     200    200     200   200
                  valid         5     5       5      5       5     5

Aucune image concernant le Plush Crested Jay n'est à la bonne taille. On se propose donc de laisser tomber cette espèce.
En revanche, le Loggerhead Shrike n'a qu'une image qui pose problème. On va voir si on peut la garder en changeant sa taille. Sinon on pourra la supprimer car c'est une parmi 200

In [11]:
imageToResize = df_to_resize[df['birdName']== "LOGGERHEAD SHRIKE"]
ratio = float(imageToResize['height']/imageToResize['width'])
print("Le ratio de la hauteur sur la largeur et de %.2f : c'est assez proche de 1 pour la redimensionner sans perdre trop d'information" % ratio)

Le ratio de la hauteur sur la largeur et de 1.15 : c'est assez proche de 1 pour la redimensionner sans perdre trop d'information


In [10]:
#Resize l'image de Loggerhead Shrike
img_path = os.path.join(const.DATASET_PATH, imageToResize['set'].iloc[0], imageToResize['birdName'].iloc[0], imageToResize['filename'].iloc[0])
img = Image.open(img_path)
img_resize = img.resize((224,224))
img_resize_path = os.path.join(const.DATASET_CLEAN_PATH, imageToResize['set'].iloc[0], imageToResize['birdName'].iloc[0], imageToResize['filename'].iloc[0])
img_resize.save(img_resize_path)


In [9]:
#On va essayer de voir si la classe "PLUSH CRESTED JAY" peut être facilement conservée
df_PCJ = df_to_resize[df_to_resize['birdName'] == "PLUSH CRESTED JAY"]
df_PCJ['ratio_size'] = np.abs(df_PCJ['height'] / df_PCJ['width'])
df_PCJ['ratio_size_close_to_1'] = 1 - df_PCJ['ratio_size'] < 0.2 #On décide que si l'écart de ratio par rapport à 1 est de plus de 20%, le redimensionnement fait perdre trop d'info
print(df_PCJ['ratio_size_close_to_1'].value_counts())
#Il y a trop peu d'images proches d'un carré et donc facilement resizable, on confirme la suppression de la classe "PLUSH CRESTED JAY"

ratio_size_close_to_1
True     163
False     47
Name: count, dtype: int64


In [66]:
df['mode'].value_counts()
#Toute les images sont en couleurs

mode
RGB    89886
Name: count, dtype: int64

In [67]:
#Fonction permettant de supprimer dans un dataset toute les classes d'un DF
def delClasses(df, dataset_path = const.DATASET_CLEAN_PATH):
    for dir in os.listdir(dataset_path):
        for birdName in df['birdName'].unique():
            pathToDel = os.path.join(dataset_path, dir, birdName)
            if os.path.isdir(pathToDel):
                shutil.rmtree(pathToDel)
delClasses(df_PCJ)

In [68]:
#Génération du csv de metadata pour le dossier clean
generate_metadata_csv("image_cleaned.csv", const.DATASET_CLEAN_PATH)

In [70]:
#Vérification qu'il n'y a plus d'image de dimension différente de (224,224)
df_cleaned = pd.read_csv(os.path.join(const.DATA_PATH, "image_cleaned.csv"))
df_to_resize = df_cleaned[df_cleaned['size'] != '(224, 224)']
df_to_resize.head()

,set,birdName,filename,size,height,width,format,mode
